# Statistics
## CCS 1 - Week 5

In this notebook, we outline how to conduct some common statistical analyses in Python. 

As outlined in the lecture, it's always good to combine visulalizations with statistical analysis and not just blindly calculate some statistics without understanding the data. Therefore, for consistency, we'll use the same datasets we used in the visualization examples.

In [ ]:
import pandas as pd

import seaborn as sns               # for the dataset

from scipy.stats import pearsonr    # for bivariate correlations
from scipy.stats import contingency # for crosstab statistics
import pingouin as pg               # for t-tests
import numpy as np                  # always handy but now needed for t-tests
from statsmodels.formula.api import ols  # for regression

In [ ]:
titanic = sns.load_dataset("titanic")
titanic.head()

# 1. Univariate statistics

In [ ]:
titanic.describe()

In [ ]:
titanic['class'].value_counts()

In [ ]:
# You want percentages instead? No problem, just set normalize=True:
titanic['class'].value_counts(normalize=True)

# 2. Bivariate statistics

## 2.1 Continous variables

In [ ]:
# Let us first create a temporary dataset without missings on the variables we care about
# That's because we cannot calculate a correlation if we have missings.

df = titanic[['fare','age']].dropna()
print(f"We deleted {len(titanic)-len(df)} of the originally {len(titanic)} rows.")
print(f"We created a new dataset df with only the columns fare and age and no missing values.")

In [ ]:
# The test itself is just one line - it returns a tuple with the correlation and its p-value)
pearsonr(df['fare'], df['age'])

In [ ]:
# or nicely formatted with an f-string:
r, p = pearsonr(df['fare'], df['age'])
print(f"The correlation between the variables age and fare is r={r:.3f}, p={p:.3f}.")

## 2.2 Nominal variables

In [ ]:
#let's look at the crosstab
mycrosstab = pd.crosstab(titanic['sex'], titanic['class'])
mycrosstab


In [ ]:
# and then get the chi2 test for that table
c, p, dof, expected = contingency.chi2_contingency(mycrosstab)
print(f"𝜒²({dof:.0f}) = {c:.3f}, p = {p:.3f} ")

# maybe Cramer's V as well?
print(f"V = {contingency.association(mycrosstab,method='cramer'):.3f}")

# 3. Hypothesis testing and group comparisons

## 3.1 Two groups: t-test
T-tests can be a simple way to compare the means of two groups with each other. It's actually not that popular to do t-tests in pandas, I guess because they
- are mathematically equivalent to a regression with a dummy variable for the group and therefore not really needed
- very often, one may want to control for other things anyway

In any event, there is no convenient way to do a t-test out of the box on a pandas dataframe that would conform to the reporting guidelines you learned in your statitics classes, therefore I wrote a function that does so.
You do **not** have to change the function, you can just copy-paste it if needed. Then, you can just use the function with one line.

In [ ]:
# Damian's function to provide a simple way to display the results of a t-test
# Note: pg.ttest() automatically checks for equality of variances (see pg.ttest? )
# so you don't have to do this manually. Take that, SPSS!!!

def pandasttest(df, dv, between):
    '''Takes a dataframe, the name of the column that indicates the group (`between`), 
    and the name of the column that contains the dependent variable (`dv`) as
    input and then prints a t-test'''
    
    assert len(df[between].unique())==2, "There are more than two groups, aborting"
             
    (label1, values1), (label2, values2) = df.groupby(between)[dv]
    testresult = pg.ttest(values1, values2)
    display(testresult)
    print(f"The difference between group {label1} (M={np.mean(values1):.2f}, SD={np.std(values1):.3f}) "
         f"and group {label2} (M={np.mean(values2):.2f}, SD={np.std(values2):.3f}) "
         f"is {'not ' if not (testresult['p-val']<.05).iloc[0] else''}significant, "
         f"t({testresult.iloc[0,1]})={testresult.iloc[0,0]:.3f}, "
         f"p={testresult.iloc[0,3]:.4f}.")



In [ ]:
# As you see, with the functions below
pandasttest(titanic, dv = 'fare', between = 'survived')

## 3.2 More groups: ANOVA

This is simple:

In [ ]:
aov = pg.anova(data=titanic, dv='fare', between='class', detailed=True)
aov

In [ ]:
# post-hoc test
titanic.pairwise_tukey(dv='fare', between='class').round(3)

### 3.2.1 ANOVA with interaction

In [ ]:
titanic.anova(dv="fare", between=["class", "sex"],effsize="n2").round(3)

In [ ]:
# Let's visualize this:
sns.catplot(x='class', y='fare', hue='sex', kind='bar', data=titanic)

### 3.2.1 ANCOVA
We can also add one or more covariates:

In [ ]:
pg.ancova(data=titanic, dv='fare', covar=['age'], between='class',effsize="n2")

# 4. Regression
Estimating a regression model is easy, we just need to specify the formula as follows:
`dependent_variable ~ indepent_variable_1 + independent_variable_2 + independent_variable_x ....`

In [ ]:
ols("fare ~ age + sex", data=titanic).fit().summary()

## 4.1 Mediation Analysis
It's acually astonishlingly easy to test a mediation model in Python. Note that here, it doesn't make much sense theoretically (and you shouldn't do this with a binary dependent variable), but this is how it looks like if we estimate the model  `age --> fare --> survival`:

In [ ]:
pg.mediation_analysis(data=titanic, x='age', m='fare', y='survived', seed=42, n_boot=1000)